<a href="https://colab.research.google.com/github/JericN/rice-disease-classifier/blob/main/results_inference_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Install Libraries

In [ ]:
! pip install --quiet -U fvcore
! pip install --quiet tqdm

### Import Libraries

In [ ]:
import os
import torch
import pandas as pd
from transformers import AutoModelForImageClassification
from fvcore.nn import FlopCountAnalysis
from tqdm import tqdm

### Define Test Config

In [ ]:
# Define the output path in Google Drive
root_dir = "/content/drive/Shareddrives/CS198-Drones/"
output_path = '[TESTv5] Results/model_benchmark.xlsx'

# Define the list of model names or paths from Hugging Face
model_names = [
    "SodaXII/mobilevit-small_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/vit-base-patch16-224_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/mobilevitv2_150.cvnets_in22k_ft_in1k_rice-leaf-disease-augmented-v4_v5_fft",
]

# Define batch size and number of iterations
batch_size = 64
num_iterations = 100

### Get Model Inference Data

In [ ]:
def evaluate_model(model_name, batch_size=64, num_iterations=100):
    try:
        # Initialize model and move to GPU
        model = AutoModelForImageClassification.from_pretrained(model_name).to("cuda")
        model.eval()

        # Create a dummy input tensor with the specified batch size and move it to GPU
        input_tensor = torch.randn(batch_size, 3, 224, 224).to("cuda")

        # Warm-up iterations
        for _ in range(20):
            with torch.no_grad():
                _ = model(input_tensor)

        # Reset peak memory statistics before inference
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

        # Create CUDA events for timing
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)

        # Record the start event
        start_event.record()

        # Run inference multiple times
        with torch.no_grad():
            for _ in range(num_iterations):
                _ = model(input_tensor)

        # Record the end event
        end_event.record()

        # Wait for the events to be recorded
        torch.cuda.synchronize()

        # Calculate average inference time per batch in milliseconds
        total_time_ms = start_event.elapsed_time(end_event)
        average_inference_time_ms = total_time_ms / num_iterations

        # Retrieve peak memory usage
        peak_memory = torch.cuda.max_memory_allocated()

        # Perform a forward pass to collect FLOP counts
        flop_counts = FlopCountAnalysis(model, input_tensor)
        total_flops = flop_counts.total()

        # Return the evaluation metrics
        return {
            "Model": model_name,
            "FLOPs": total_flops,
            "Average Inference Time (ms)": average_inference_time_ms,
            "Peak GPU Memory Usage (MB)": peak_memory / (1024 ** 2)
        }

    except Exception as e:
        print(f"An error occurred while evaluating model {model_name}: {e}")
        return None

### Runner Function

In [ ]:
output_file = os.path.join(root_dir, output_path)
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Initialize a list to store results
results = []

# Iterate over each model and evaluate
for model_name in tqdm(model_names, desc="Evaluating Models"):
    metrics = evaluate_model(model_name, batch_size, num_iterations)
    if metrics:
        results.append(metrics)

# Create a DataFrame from the results
df_models = pd.DataFrame(results)

# Save the results to an Excel file in Google Drive
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    df_models.to_excel(writer, sheet_name="Model Info", index=False)

print(f"Benchmark results saved to: {output_file}")